In [ ]:
import sys
sys.path.append("..") # Ensure the parent directory is in the path

import core.models as Models
import core
from core.DataLoader import DataPreprocessor, DataConfig
import core.AssignmentKFold as KFold
import numpy as np
from importlib import reload
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
MAX_JETS = 4
PLOTS_DIR = f"../plots/"
import os
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

config = DataConfig(jet_features=["ordered_jet_pt", "ordered_jet_e", "ordered_jet_phi", "ordered_jet_eta", "ordered_jet_b_tag"], 
                                lepton_features=["lep_pt","lep_e", "lep_eta", "lep_phi"],
                                jet_truth_label="ordered_event_jet_truth_idx", 
                                lepton_truth_label="event_lepton_truth_idx", 
                                global_features = ["met_met_NOSYS","met_phi_NOSYS"], 
                                max_leptons=2, 
                                max_jets = MAX_JETS, 
                                non_training_features= ["truth_ttbar_mass", "truth_ttbar_pt", "N_jets"], 
                                event_weight="weight_mc_NOSYS")

DataProcessor = DataPreprocessor(config)
DataProcessor.load_data("/data/dust/group/atlas/ttreco/full_training.root", "reco", max_events=1000000)
DataProcessor.normalise_data()
DataProcessor.split_data(0.2)

In [ ]:

reload(KFold)

TransformerMatcher = Models.RNN(DataProcessor); TransformerMatcher.load_data(*DataProcessor.get_data())
TransformerMatcher.build_model(
    hidden_dim=64,
    num_heads=8,
    num_layers=4,
    dropout_rate=0.1,
)
TransformerMatcher.compile_model(
    lambda_excl=0, optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5)
)
TransformerMatcher.model.summary()
TransformerMatcher.load_data(*DataProcessor.get_data())

Model: "FeatureConcatTransformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ global_inputs       │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 2)         │          0 │ global_inputs[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 8)         │          0 │ lep_inputs[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 4, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_2     │ (None, 4, 2)      │          0 │ flatten_2[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_3     │ (None, 4, 8)      │          0 │ flatten_3[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 4, 15)     │          0 │ jet_inputs[0][0], │
│ (Concatenate)       │                   │            │ repeat_vector_2[… │
│                     │                   │            │ repeat_vector_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding (MLP) │ (None, 4, 64)     │          0 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 4, 1)      │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 4, 64)     │     33,472 │ jet_embedding[0]… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 4, 64)     │     33,472 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 4, 64)     │     33,472 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 4, 64)     │     33,472 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_outputs_mlp     │ (None, 4, 64)     │          0 │ self_attention_b… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_output_embeddi… │ (None, 4, 2)      │          0 │ jet_outputs_mlp[… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_assignment_pro… │ (None, 4, 2)      │          0 │ jet_output_embed… │
│ (TemporalSoftmax)   │                   │            │ jet_mask[0][0]    │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 133,888 (523.00 KB)

 Trainable params: 133,888 (523.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
TransformerMatcher.train_model(epochs=100,
                                batch_size=512,
                                verbose=1,
                                weight = "sample",
                                callbacks = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=50, restore_best_weights=True, mode ="max"))


Starting training...
Epoch 1/100
1555/1555 ━━━━━━━━━━━━━━━━━━━━ 240s 149ms/step - accuracy: 0.6837 - loss: 1.8270 - val_accuracy: 0.7341 - val_loss: 1.4062
Epoch 2/100
1555/1555 ━━━━━━━━━━━━━━━━━━━━ 243s 156ms/step - accuracy: 0.7147 - loss: 1.4649 - val_accuracy: 0.7612 - val_loss: 1.2521
Epoch 3/100
1555/1555 ━━━━━━━━━━━━━━━━━━━━ 256s 165ms/step - accuracy: 0.7302 - loss: 1.3434 - val_accuracy: 0.7764 - val_loss: 1.1780
Epoch 4/100
1555/1555 ━━━━━━━━━━━━━━━━━━━━ 251s 161ms/step - accuracy: 0.7399 - loss: 1.2761 - val_accuracy: 0.7742 - val_loss: 1.1272
Epoch 5/100
1555/1555 ━━━━━━━━━━━━━━━━━━━━ 255s 164ms/step - accuracy: 0.7502 - loss: 1.2321 - val_accuracy: 0.7938 - val_loss: 1.0803
Epoch 6/100
1555/1555 ━━━━━━━━━━━━━━━━━━━━ 255s 164ms/step - accuracy: 0.7590 - loss: 1.1957 - val_accuracy: 0.7885 - val_loss: 1.0383
Epoch 7/100
1555/1555 ━━━━━━━━━━━━━━━━━━━━ 219s 141ms/step - accuracy: 0.7617 - loss: 1.1651 - val_accuracy: 0.7991 - val_loss: 1.0160
Epoch 8/100
1555/1555 ━━━━━━━━━━━━

In [ ]:
TransformerMatcher.plot_history()

In [ ]:
TransformerMatcher.plot_confusion_matrix()

In [ ]:
TransformerMatcher.plot_permutation_importance(shuffle_number=5)